In [29]:
from bs4 import BeautifulSoup as bs
import requests
import os
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

In [30]:
#open chrome driver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/charlie/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


<h1 style="color:blue">Scraping first headline </h1>

In [31]:
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html


soup = bs(html, 'html.parser')

start = soup.find('li', class_="slide")
title = start.find('div', class_='content_title').text.strip()

title

'Another First: Perseverance Captures the Sounds of Driving on Mars'

<h1 style="color:blue">Scraping first teaser paragraph </h1>

In [32]:
news_start = soup.find("li", class_= "slide")
news_p = news_start.find('div', class_="article_teaser_body").text
news_p

'NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.'

<h1 style="color:blue"> JPL Mars Space Images - Featured Image </h1>

In [33]:
jpl_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
browser.visit(jpl_url)

html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information
images = soup.find_all('div', class_='floating_text_area')

for image in images:
    link = image.find("a")
    href = link['href']
    
    featured_image_url = ("https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/" + href)
    
featured_image_url
    

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg'

<h1 style="color:blue">Table Facts</h1>

In [34]:
facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(facts_url)

mars_table_df = tables[0]
mars_tableII = mars_table_df.set_index(0, inplace=True)

mars_tableIII = mars_table_df.rename(columns={0:' ',1:' '})

mars_html_table = mars_tableIII.to_html()

mars_html_table_clean = mars_html_table.replace("\n", "")

mars_html_table_clean

# mars_data['mars_html_table_clean'] = mars_html_table_clean



'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th></th>    </tr>    <tr>      <th>0</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

<h1 style="color:blue"> Mars Hemispheres </h1>

In [57]:
    hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

    browser.visit(hemi_url)

    jpg_list=browser.find_by_css("a.product-item h3")

    mars_hemi_image_urls = []

    for x in range(len(jpg_list)):
        hemi={}
        browser.find_by_css("a.product-item h3")[x].click()
        sample = browser.links.find_by_text("Sample").first
        hemi["img_url"]=sample["href"]
        hemi["title"]=browser.find_by_css("h2.title").text
        mars_hemi_image_urls.append(hemi)
        browser.back()

    print(mars_hemi_image_urls)    

    browser.quit()



NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=89.0.4389.90)


In [58]:
mars_data =({

    'title': title,
    'news_p': news_p,
    'featured_image_url': featured_image_url,
    'mars_html_table_clean': mars_html_table_clean,
    'hemi': mars_hemi_image_urls
})

mars_data


{'title': 'Another First: Perseverance Captures the Sounds of Driving on Mars',
 'news_p': 'NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.',
 'featured_image_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg',
 'mars_html_table_clean': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th></th>    </tr>    <tr>      <th>0</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1

In [59]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [60]:
db = client.marsData_db

In [61]:
marsData = db.marsData

In [62]:
marsData.insert_one(mars_data)

In [64]:
mars_data["hemi"][0]['img_url']

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'